In [ ]:
import math

from PIL import Image
import requests
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina'

import ipywidgets as widgets
from IPython.display import display, clear_output

import torch
from torch import nn
from torchvision.models import resnet50
import torchvision.transforms as T
torch.set_grad_enabled(False);

CLASSES = [
    'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
    'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
    'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
    'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
]

COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

COLORS *= 100

transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
def rescale(box, size):
    img_w, img_h = size
    x, y, w, h = box.unbind(1)
    result = [(x - 0.5 * w), (y - 0.5 * h), (x + 0.5 * w), (y + 0.5 * h)]
    result = torch.stack(result, dim=1)
    result *= torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return result


def plot(image, predictions, scaled):
    plt.figure(figsize=(16,10))
    plt.imshow(image)
    axes = plt.gca()
    for pred, (xmin, ymin, xmax, ymax), color in zip(predictions, scaled.tolist(), COLORS):
        axes.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin, fill=False, color=color, linewidth=3))
        text = f"{CLASSES[pred.argmax()]}: {pred[pred.argmax()]:0.2f}"
        axes.text(xmin, ymin, text, fontsize=12, bbox=dict(facecolor="orange", alpha=0.5))
    plt.axis("off")
    plt.show()

In [ ]:
model = torch.hub.load("facebookresearch/detr", "detr_resnet50", pretrained=True)
model.eval();

In [ ]:
urls = ["http://images.cocodataset.org/val2017/000000039769.jpg",
        "https://farm7.staticflickr.com/6096/6225161122_20e1b36fc0_z.jpg",
        "https://farm7.staticflickr.com/6135/5996911372_783d085b50_z.jpg",
        "https://farm9.staticflickr.com/8302/7877972164_51f717def0_z.jpg",
        "https://farm7.staticflickr.com/6168/6148670625_1caaf41c72_z.jpg",
        "https://farm6.staticflickr.com/5457/9641772821_5b20a20068_z.jpg"]

downloaded_images = [Image.open(requests.get(url, stream=True).raw) for url in urls]

In [ ]:
def run_image(downloaded_image, threshold_up = 1, threshold_down = 0.7):
    outputs = model(transform(downloaded_image).unsqueeze(0))
    predictions = outputs['pred_logits'].softmax(-1)[0, :, :-1]
    best_predictions_1 = predictions.max(-1).values >= threshold_down
    best_predictions_2 = predictions.max(-1).values <= threshold_up
    best_predictions = torch.logical_and(best_predictions_1, best_predictions_2)
    scaled = rescale(outputs['pred_boxes'][0, best_predictions], downloaded_image.size)
    plot(downloaded_image, predictions[best_predictions], scaled)


for downloaded_image in downloaded_images:
    run_image(downloaded_image)